In [1]:
import pandas as pd
df = pd.read_csv("Seasons_Stats.csv")

In [33]:
df.sort_values(by=["Year"], ascending=False)

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG
24690,24690,2017.0,Ivica Zubac,C,19.0,LAL,38.0,11.0,609.0,17.0,...,41.0,118.0,159.0,30.0,14.0,33.0,30.0,66.0,284.0,7.473684
24299,24299,2017.0,Gerald Green,SF,31.0,BOS,47.0,0.0,538.0,12.0,...,17.0,68.0,85.0,33.0,9.0,7.0,25.0,48.0,262.0,5.574468
24297,24297,2017.0,Danny Green,SG,29.0,SAS,68.0,68.0,1807.0,10.2,...,31.0,193.0,224.0,124.0,71.0,58.0,76.0,120.0,497.0,7.308824
24296,24296,2017.0,Jerian Grant,PG,24.0,CHI,63.0,28.0,1028.0,13.1,...,17.0,94.0,111.0,120.0,47.0,8.0,44.0,93.0,370.0,5.873016
24295,24295,2017.0,Jerami Grant,SF,22.0,OKC,78.0,4.0,1490.0,10.3,...,38.0,161.0,199.0,46.0,33.0,75.0,41.0,144.0,421.0,5.397436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21678,21678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22252,22252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22864,22864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23516,23516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [3]:
df.columns

Index(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [5]:
len(df.columns)

53

In [6]:
len(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS', 'WS/48', 'blank2',
       'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'])

53

In [7]:
df['PPG'] = df.apply(lambda row: row.PTS / row.G, axis=1)

In [8]:
df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0,7.269841
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0,5.693878
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0,6.537313
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0,4.200000
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0,4.538462


In [9]:
len(df)

24691

In [41]:
df[df["Player"] == "Dahntay Jones"]

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PPG
16764,16764,2004.0,Dahntay Jones,SG,23.0,MEM,20.0,0.0,154.0,2.5,...,7.0,16.0,23.0,12.0,5.0,6.0,12.0,23.0,36.0,1.800000
17360,17360,2005.0,Dahntay Jones,SG,24.0,MEM,52.0,7.0,649.0,9.6,...,11.0,58.0,69.0,21.0,13.0,11.0,29.0,81.0,233.0,4.480769
17934,17934,2006.0,Dahntay Jones,SF,25.0,MEM,71.0,4.0,967.0,8.0,...,20.0,84.0,104.0,39.0,38.0,15.0,50.0,108.0,281.0,3.957746
18479,18479,2007.0,Dahntay Jones,SG,26.0,MEM,78.0,25.0,1671.0,9.6,...,47.0,106.0,153.0,68.0,39.0,22.0,92.0,192.0,583.0,7.474359
19043,19043,2008.0,Dahntay Jones,SG,27.0,SAC,25.0,0.0,206.0,13.5,...,7.0,28.0,35.0,12.0,8.0,6.0,10.0,28.0,81.0,3.240000
19630,19630,2009.0,Dahntay Jones,SG,28.0,DEN,79.0,71.0,1426.0,9.0,...,58.0,110.0,168.0,78.0,49.0,19.0,68.0,199.0,429.0,5.430380
20226,20226,2010.0,Dahntay Jones,SF,29.0,IND,76.0,26.0,1892.0,10.8,...,48.0,181.0,229.0,150.0,40.0,39.0,129.0,237.0,776.0,10.210526
20838,20838,2011.0,Dahntay Jones,SG,30.0,IND,45.0,2.0,589.0,13.7,...,4.0,61.0,65.0,33.0,20.0,7.0,31.0,70.0,283.0,6.288889
21421,21421,2012.0,Dahntay Jones,SF,31.0,IND,65.0,3.0,1052.0,10.3,...,23.0,94.0,117.0,63.0,23.0,11.0,46.0,119.0,346.0,5.323077
21954,21954,2013.0,Dahntay Jones,SF,32.0,TOT,78.0,19.0,1016.0,6.4,...,24.0,77.0,101.0,48.0,22.0,6.0,38.0,119.0,262.0,3.358974


In [58]:
df_filtered = df[df["G"] > 10]
df_filtered = df_filtered[df_filtered["PPG"] > 15]

In [59]:
df_age_ppg = df_filtered.groupby('Player').apply(lambda x: x.loc[x.PPG == x.PPG.max(),['Player','Age', "PPG", "Year"]])
# df_age_ppg = df.groupby('Player').apply(lambda x: x.loc[x.PPG == x.PPG.max()])

In [60]:
df_age_ppg_year = df_age_ppg.sort_values(by=["Age"], ascending=False).dropna()

In [61]:
df_age_ppg_year_100 = df_age_ppg_year.head(100)

In [62]:
import numpy as np
df_age_ppg_year_100["Year"] = df_age_ppg_year_100["Year"].apply(np.int64)

<ipython-input-62-d52d44914372>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_ppg_year_100["Year"] = df_age_ppg_year_100["Year"].apply(np.int64)


In [63]:
df_age_ppg_year_100["Age"] = df_age_ppg_year_100["Age"].apply(np.int64)

<ipython-input-63-ad3735108c52>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_ppg_year_100["Age"] = df_age_ppg_year_100["Age"].apply(np.int64)


In [64]:
df_age_ppg_year_100 = df_age_ppg_year_100.round(1)

In [65]:
df_age_ppg_year_100.head()

,,Player,Age,PPG,Year
Player,,,,,
Sam Cassell,16594,Sam Cassell,34,19.8,2004
Arvydas Sabonis*,13820,Arvydas Sabonis*,33,16.0,1998
Chauncey Billups,19973,Chauncey Billups,33,19.5,2010
Frank Brickowski,10961,Frank Brickowski,33,16.9,1993
Connie Dierking,2908,Connie Dierking,33,16.7,1970


In [66]:
df_age_ppg_year_100.to_json(orient="records")

'[{"Player":"Sam Cassell","Age":34,"PPG":19.8,"Year":2004},{"Player":"Arvydas Sabonis*","Age":33,"PPG":16.0,"Year":1998},{"Player":"Chauncey Billups","Age":33,"PPG":19.5,"Year":2010},{"Player":"Frank Brickowski","Age":33,"PPG":16.9,"Year":1993},{"Player":"Connie Dierking","Age":33,"PPG":16.7,"Year":1970},{"Player":"John Lucas","Age":33,"PPG":17.5,"Year":1987},{"Player":"Detlef Schrempf","Age":32,"PPG":19.2,"Year":1995},{"Player":"Toni Kukoc","Age":32,"PPG":19.7,"Year":2001},{"Player":"Hakeem Olajuwon*","Age":32,"PPG":27.8,"Year":1995},{"Player":"Alex English*","Age":32,"PPG":29.8,"Year":1986},{"Player":"Mike Newlin","Age":32,"PPG":21.4,"Year":1981},{"Player":"Marc Gasol","Age":32,"PPG":19.5,"Year":2017},{"Player":"Bob Davies*","Age":32,"PPG":16.2,"Year":1952},{"Player":"Bob Boozer","Age":31,"PPG":21.7,"Year":1969},{"Player":"Bill Sharman*","Age":31,"PPG":22.3,"Year":1958},{"Player":"Darrell Armstrong","Age":31,"PPG":16.2,"Year":2000},{"Player":"Chet Walker*","Age":31,"PPG":22.0,"Year":